<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/NN_from_scratch_corso_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class LinearLayer:
    def __init__(self, n_in, n_out):
        # inizializzare con distribuzione normale standard
        self.W = None
        # inizializzare a 0
        self.b = None

        # hanno le stesse dimensioni di W e di b
        self.dW = None
        self.db = None

    def forward(self, x):
        ''' restituisce y = Wx + b

        args
        ----
        x (np.array): x.shape = (n_in, 1)

        ret
        ---
        y (np.array): y.shape = (n_out, 1)
        '''
        pass

    def backward(self, dy):
        ''' restituisce dx (dl/dx) e valorizza
        gli attributi dW e db
        
        args
        ----
        dy (np.array): dy.shape (n_out, 1)

        ret
        ---
        dx (np.array): dx.shape = (n_in, 1)
        '''
        pass
    